In [ ]:
import xarray as xr

# city="barcelona"
# city="perugia"
# city="trento"
# city="roma"
# product_type = "L0-LST-TD-FM"
# product_type = "L0-LST-TN-FM"
# INPUT_ZARR_FILE_PATH = f"products/{city}/{product_type}-{city}.zarr"
# INPUT_ZARR_FILE_PATH = "products/perugia/new_algo/L0-LST-TD-FM-perugia.zarr"
# INPUT_ZARR_FILE_PATH = "products/perugia/old_products/L0-LST-TD-FM-perugia.zarr"
INPUT_ZARR_FILE_PATH = "/mnt/c/Users/AlviseFerrari/Documents/GMATICS_laptop/DTC/Task_6_2/Landsat/L0-LST-TD-FM-pesaro.zarr"

ds = xr.open_zarr(INPUT_ZARR_FILE_PATH)
ds

In [ ]:
from pathlib import Path

out_dir = Path("tiffs")
out_dir.mkdir(exist_ok=True)

In [ ]:
import pandas as pd

# seleziona solo i mesi 2024-06 … 2024-09 (inclusi)
# sub_ds = ds.sel(time=slice("2024-06", "2024-09"))
# sub_ds = ds.sel(time=slice("2025-06", "2025-09"))
sub_ds = ds.sel(time=slice("2025-07"))

for ts in sub_ds.time:
    month_ds = sub_ds.sel(time=ts)

    stacked = xr.concat(
        [month_ds["temp"]],
        dim="band",
    ).transpose("band", "y", "x")

    stacked = stacked.assign_coords(band=("band", [1]))
    stacked = stacked.rio.write_coordinate_system()

    out_path = out_dir / f"{product_type}-{city}-{pd.to_datetime(ts.values):%Y_%m}.tif"
    stacked.rio.to_raster(
        out_path,
        driver="COG",
        BLOCKSIZE=512,
        compress="DEFLATE",
        NUM_THREADS="ALL_CPUS",
        overview_resampling="average",
    )



In [ ]:
import xarray as xr
import rioxarray  # attiva accessor .rio
import rasterio
import numpy as np
import pandas as pd
import json

# 1) Sottoselezione temporale (inclusivo)
sub_ds = ds.sel(time=slice("2025-06", "2025-09"))

# 2) Prendi la variabile da salvare (qui 'temp') e riordina le dimensioni in (time, y, x)
da = sub_ds["temp"].transpose("time", "y", "x")

# 3) Crea uno stack multibanda: una banda per ciascun timestamp
#    - rinominiamo 'time' -> 'band' e assegniamo indici banda 1..N
stacked = (
    da.rename({"time": "band"})
      .assign_coords(band=np.arange(1, da.sizes["time"] + 1))
      .rio.write_coordinate_system()
)

# 4) Percorso di output (un unico file per l’intervallo richiesto)
t0 = pd.to_datetime(sub_ds.time.values[0])
t1 = pd.to_datetime(sub_ds.time.values[-1])
out_path = out_dir / f"{product_type}-{city}-{t0:%Y_%m}-{t1:%Y_%m}.tif"

# 5) Scrittura GeoTIFF "normale" (NO COG)
#    - lasciamo Raster tiling/overview disattivati per evitare COG-like
#    - compressione opzionale
stacked.rio.to_raster(
    out_path,
    driver="GTiff",
    compress="DEFLATE",
    BIGTIFF="IF_SAFER",   # utile se il file cresce
)

# 6) Scrivi metadati temporali per ogni banda + tag globale
times = pd.to_datetime(sub_ds.time.values)
iso_list = [pd.to_datetime(ts).strftime("%Y-%m-%dT%H:%M:%S") for ts in times]

with rasterio.open(out_path, "r+") as dst:
    # per-banda: description e tag TIMESTAMP
    for i, iso in enumerate(iso_list, start=1):
        dst.set_band_description(i, iso)        # visibile come descrizione banda
        dst.update_tags(i, TIMESTAMP=iso)       # tag per-banda

    # tag globale: elenco JSON di tutti i timestamp
    dst.update_tags(TIMESTAMPS=json.dumps(iso_list))

print(f"Salvato GeoTIFF multibanda con {len(iso_list)} bande:", out_path)